In [1]:
import os, json
import pandas as pd

In [2]:
years = [2016,2022]
dfs_dfs = []
for year in years:
    path_str = 'data/DFS_Salaries/' + str(year) + '/'
    for x, file_name in enumerate([x_json for x_json in os.listdir(path_str) if x_json.endswith('.json')]):
        with open(path_str + file_name) as json_file:
            if x == 0:
                dfs_df = pd.DataFrame(json.load(json_file))
            else:
                dfs_df = pd.concat([dfs_df,pd.DataFrame(json.load(json_file))])
    dfs_df['Season'] = year
    dfs_dfs.append(dfs_df)

In [3]:
#combine all years of dataframes
dfs_df = pd.concat(dfs_dfs)

In [45]:
#create a unique id for grouping
dfs_df['grouping'] = dfs_df['Team']+dfs_df['Position']+dfs_df['Season'].astype(str)+dfs_df['Week'].astype(str).str.zfill(2)

In [33]:
#inverting salary to groupby "first" which relies on minimum instead of maximum
dfs_df['inv_sal'] = 1/dfs_df['DraftKingsSalary']

In [46]:
#get ranking of minimum inverse salaries by team/position/week/season which is the same as ranking of descending max salaries
dfs_df['pos_team_rank_week'] = dfs_df.groupby('grouping')['inv_sal'].rank(method = 'first')

In [47]:
#create a new column to ensure no duplicates occured when ranking
dfs_df['dup_test'] = dfs_df['grouping'] + dfs_df['pos_team_rank_week'].astype(str)

In [55]:
dfs_df.to_csv('test_sals.csv')

In [52]:
#ensuring all non-NaN values are not duplicated
dfs_df[dfs_df['DraftKingsSalary'].notna()].duplicated(subset = ['dup_test']).unique()

array([False])

In [60]:
dfs_df['pos_rank'] = dfs_df['Position'] + dfs_df['pos_team_rank_week'].astype(str)

In [115]:
#make new dataframe for testing
dfs_df_test = dfs_df.dropna(axis = 0, subset = ['DraftKingsSalary']).reset_index(drop=True)

In [116]:
dfs_df_test

,Name,Team,Position,Week,Opponent,FantasyPointsDraftKings,UpcomingOpponentRank,UpcomingOpponentPositionRank,DraftKingsSalary,Rank,Season,grouping,pos_team_rank_week,dup_test,inv_sal,pos_rank
0,Seattle Seahawks,SEA,DST,15,LAR,11.9,None,None,4000.0,1,2016,SEADST201615,1.0,SEADST2016151.0,0.000250,DST1.0
1,Seattle Seahawks,SEA,DST,2,LAR,11.5,None,None,3800.0,2,2016,SEADST201602,1.0,SEADST2016021.0,0.000263,DST1.0
2,Philadelphia Eagles,PHI,DST,9,NYG,10.6,None,None,3100.0,3,2016,PHIDST201609,1.0,PHIDST2016091.0,0.000323,DST1.0
3,Seattle Seahawks,SEA,DST,1,MIA,10.5,None,None,3900.0,4,2016,SEADST201601,1.0,SEADST2016011.0,0.000256,DST1.0
4,Arizona Cardinals,ARI,DST,6,NYJ,10.5,None,None,3800.0,4,2016,ARIDST201606,1.0,ARIDST2016061.0,0.000263,DST1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16418,Davion Davis,HOU,WR,7,LV,0.0,28.0,21.0,3000.0,1207,2022,HOUWR202207,10.0,HOUWR20220710.0,0.000333,WR10.0
16419,Garrett Gilbert,NE,QB,7,CHI,0.0,9.0,11.0,4800.0,1207,2022,NEQB202207,3.0,NEQB2022073.0,0.000208,QB3.0
16420,Andy Isabella,BAL,WR,7,CLE,0.0,24.0,23.0,3000.0,1207,2022,BALWR202207,9.0,BALWR2022079.0,0.000333,WR9.0
16421,Keelan Doss,LAC,WR,7,SEA,0.0,29.0,7.0,3000.0,1207,2022,LACWR202207,10.0,LACWR20220710.0,0.000333,WR10.0


In [117]:
#cast as int then str to remove decimal
dfs_df_test['pos_team_rank_week'] = dfs_df_test['pos_team_rank_week'].astype(int).astype(str)

In [118]:
dfs_df_test['pos_rank'] = dfs_df_test['Position'] + dfs_df_test['pos_team_rank_week']

In [119]:
dfs_df_test

,Name,Team,Position,Week,Opponent,FantasyPointsDraftKings,UpcomingOpponentRank,UpcomingOpponentPositionRank,DraftKingsSalary,Rank,Season,grouping,pos_team_rank_week,dup_test,inv_sal,pos_rank
0,Seattle Seahawks,SEA,DST,15,LAR,11.9,None,None,4000.0,1,2016,SEADST201615,1,SEADST2016151.0,0.000250,DST1
1,Seattle Seahawks,SEA,DST,2,LAR,11.5,None,None,3800.0,2,2016,SEADST201602,1,SEADST2016021.0,0.000263,DST1
2,Philadelphia Eagles,PHI,DST,9,NYG,10.6,None,None,3100.0,3,2016,PHIDST201609,1,PHIDST2016091.0,0.000323,DST1
3,Seattle Seahawks,SEA,DST,1,MIA,10.5,None,None,3900.0,4,2016,SEADST201601,1,SEADST2016011.0,0.000256,DST1
4,Arizona Cardinals,ARI,DST,6,NYJ,10.5,None,None,3800.0,4,2016,ARIDST201606,1,ARIDST2016061.0,0.000263,DST1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16418,Davion Davis,HOU,WR,7,LV,0.0,28.0,21.0,3000.0,1207,2022,HOUWR202207,10,HOUWR20220710.0,0.000333,WR10
16419,Garrett Gilbert,NE,QB,7,CHI,0.0,9.0,11.0,4800.0,1207,2022,NEQB202207,3,NEQB2022073.0,0.000208,QB3
16420,Andy Isabella,BAL,WR,7,CLE,0.0,24.0,23.0,3000.0,1207,2022,BALWR202207,9,BALWR2022079.0,0.000333,WR9
16421,Keelan Doss,LAC,WR,7,SEA,0.0,29.0,7.0,3000.0,1207,2022,LACWR202207,10,LACWR20220710.0,0.000333,WR10


In [123]:
dfs_df2 = dfs_df_test.pivot_table(index = ['Team','Season','Week','Opponent'], columns = 'pos_rank',values = 'DraftKingsSalary',aggfunc = 'sum').reset_index()

In [126]:
dfs_df_final = dfs_df2[['Team','Season','Week','Opponent','DST1','QB1','RB1','RB2','WR1','WR2','WR3','WR4','TE1','TE2']]

In [129]:
dfs_df_final.to_csv('data/dfs_salaries.csv')

In [130]:
dfs_df_final['Team'].unique()

array(['ARI', 'ATL', 'BAL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'DAL',
       'DEN', 'DET', 'GB', 'HOU', 'IND', 'JAX', 'KC', 'LAC', 'LAR', 'LV',
       'MIA', 'MIN', 'NE', 'NO', 'NYG', 'NYJ', 'PHI', 'PIT', 'SEA', 'SF',
       'TB', 'TEN', 'WAS'], dtype=object)